In [104]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import pickle
import os
os.chdir('C:/Users/Anna/Documents/lab_at_dc_part_a/')

In [117]:
# reading in the data from csv
stops1 = pd.read_csv('stopsData1.csv', index_col='stop_id')
stops2 = pd.read_csv('stopsData2.csv', index_col='stop_id')

In [118]:
# concatting the files
stops = stops1.join(stops2, how='left')

In [119]:
# getting total traffic stops (and checking join)
print len(stops1),  len(stops2), len(stops)

2140 2140 2140


In [120]:
stops.head()

,officer_id,driver_id,driver_age,driver_sex,ticket,ward
stop_id,,,,,,
7406,B,3566,33,m,0,1
3364,A,9243,28,m,1,7
1934,D,1296,66,m,0,3
2807,E,14277,28,m,0,8
9974,D,2493,55,m,0,2


In [121]:
#figuring out average age by district
for i in np.arange(1,9,1):
    print 'Ward {}: Average Age {}'.format(i, np.average(stops[stops.ward == i]['driver_age']).round(1)) 

Ward 1: Average Age 44.9
Ward 2: Average Age 44.9
Ward 3: Average Age 46.9
Ward 4: Average Age 45.6
Ward 5: Average Age 46.0
Ward 6: Average Age 49.9
Ward 7: Average Age 48.4
Ward 8: Average Age 50.9


In [122]:
# number of distinct drivers stopped
len(stops.groupby('driver_id'))

2034

In [123]:
# dedupe driver id to see what average age of distinct drivers is
drivers = stops.drop_duplicates(subset = 'driver_id')

In [124]:
# check average age overall and average age of deduped data (do we want to double-count ppl
# if they're stopped multiple times?)
print np.average(drivers['driver_age']), 
np.average(stops['driver_age'])

47.151425762

47.118224299065417

In [125]:
# what proportion are men
len(stops[stops.driver_sex == 'm']) / len(stops)

0.6032710280373832

In [126]:
# linear regression time!
from sklearn.linear_model import LinearRegression
lr = LinearRegression(fit_intercept=True)

In [127]:
# create dummy var for male
stops['sex_m'] = None
stops.ix[stops.driver_sex == 'm', 'sex_m'] = 1
stops.ix[stops.driver_sex == 'f', 'sex_m'] = 0

In [130]:
# create X and y
X = stops[['driver_age', 'sex_m']]
y = stops.ticket

In [131]:
lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [138]:
print 'Coefficients: {}\nIntercept: {}'.format(lr.coef_, lr.intercept_)

Coefficients: [ 0.00388807  0.1752491 ]
Intercept: 0.0405177544448


In [147]:
# all else equal, age + 10
prob = (lr.intercept_ + (lr.coef_[:1] * 10))  - (lr.intercept_ + lr.coef_[:1])
print prob

[ 0.0349926]


In [151]:
## 30 yo female
prob = (lr.intercept_ + (lr.coef_[:1] * 30) + lr.coef_[1:2] * 0)
print prob

[ 0.15715975]


In [153]:
## 30 yo male vs 30 yo female
prob = (lr.intercept_ + (lr.coef_[:1] * 30) + lr.coef_[1:2] * 1) - (lr.intercept_ + (lr.coef_[:1] * 30) + lr.coef_[1:2] * 0)
print prob

[ 0.1752491]
